In [1]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss, GroupNormalizer
from lightning.pytorch.tuner import Tuner
import torch
from pytorch_forecasting.metrics import MAE
import pandas as pd

In [2]:
data = pd.read_csv('Out_120.csv')

In [46]:
# data.head()
data['Date'] = pd.to_datetime(data['Date'])

In [5]:
# download eth, xrp, sol data
import yfinance as yf

# download bitcoin data
start_date = "2019-08-01"
end_date = "2024-09-01"
ETH_USD = yf.download("ETH-USD", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [6]:
LTC_USD = yf.download("LTC-USD", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [13]:
# ETH_USD.head()
# ETH_USD.columns = ETH_USD.columns.droplevel(1)
LTC_USD.columns = LTC_USD.columns.droplevel(1)

In [17]:
LTC_USD.head()

Price,LTC_Price,LTC_Volume
Date,,
2019-08-01,99.179726,2953184689
2019-08-02,94.941292,2882784454
2019-08-03,94.580879,2512959054
2019-08-04,93.203781,2427608117
2019-08-05,96.834648,4134352823


In [15]:
# ETH_USD.rename(columns={'Close': 'ETH_Price', 'Volume': 'ETH_Volume'}, inplace=True)
LTC_USD.rename(columns={'Close': 'LTC_Price', 'Volume': 'LTC_Volume'}, inplace=True)

In [16]:
# ETH_USD = ETH_USD.drop(columns=['High', 'Low', 'Open'])
LTC_USD = LTC_USD.drop(columns=['High', 'Low', 'Open'])

In [21]:
LTC_USD['Date'] = pd.to_datetime(LTC_USD['Date'])
# LTC_USD.to_csv("LTC_USD.csv", index=True)

In [27]:
ETH_USD['Date'] = pd.to_datetime(ETH_USD['Date'])
# ETH_USD.to_csv("ETH_USD.csv", index=True)

In [23]:
ETH_USD = pd.read_csv('ETH_USD.csv')

In [26]:
ETH_USD.dtypes

Date           object
ETH_Price     float64
ETH_Volume      int64
dtype: object

In [28]:
Merged = LTC_USD.merge(ETH_USD, on="Date", how="outer")

In [34]:
Merged.describe()

,Date,LTC_Price,LTC_Volume,ETH_Price,ETH_Volume
count,1858,1858.000000,1858.000000,1858.000000,1858.000000
mean,2022-02-14 11:59:59.999999744,0.178252,0.111443,0.353113,0.161509
min,2019-08-01 00:00:00,0.000000,0.000000,0.000000,0.000000
25%,2020-11-07 06:00:00,0.079226,0.019205,0.071782,0.076422
50%,2022-02-14 12:00:00,0.123611,0.053573,0.349314,0.136651
75%,2023-05-24 18:00:00,0.217409,0.156083,0.538311,0.207722
max,2024-08-31 00:00:00,1.000000,1.000000,1.000000,1.000000
std,NaN,0.151198,0.135994,0.255221,0.119080


In [30]:
# Separate the DateTime columns and numeric columns
from sklearn.preprocessing import MinMaxScaler


date_columns = Merged.select_dtypes(include=['datetime64']).columns
numeric_columns = Merged.select_dtypes(include=['float64', 'int64']).columns

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale only the numeric columns
Merged[numeric_columns] = scaler.fit_transform(Merged[numeric_columns])

In [33]:
# check for missing values
BTC_USD_Missing = Merged.isnull().any().any()
print(f"Are there any missing values? {BTC_USD_Missing}")

Are there any missing values? False


In [43]:
Merged.head()

,Date,LTC_Price,LTC_Volume,ETH_Price,ETH_Volume
27,2019-08-28,0.102652,0.154545,0.013460,0.056006
28,2019-08-29,0.093292,0.166255,0.012530,0.061260
29,2019-08-30,0.093953,0.126259,0.012385,0.044840
30,2019-08-31,0.094637,0.120992,0.013158,0.045492
31,2019-09-01,0.099049,0.117904,0.012980,0.042150


In [38]:
# remove unwanted dates - head
start_date = '2019-08-01'
end_date = '2019-08-27'

Merged = Merged[~Merged['Date'].between(start_date, end_date)]

In [41]:
# remove unwanted dates - tail
start_date = '2024-08-24'
end_date = '2024-08-31'

Merged = Merged[~Merged['Date'].between(start_date, end_date)]

In [47]:
# merge with the main dataset
Merged_Final = Merged.merge(data, on="Date", how="outer")

In [49]:
data.head()

,Unnamed: 0,Date,BTC_Price,BTC_Volume,Gold_Price,Interest_Rate,VIX_Close
0,0,2019-08-28,0.070231,0.035507,0.079985,0.393195,0.109768
1,1,2019-08-29,0.066646,0.033893,0.071482,0.393195,0.089108
2,2,2019-08-30,0.067937,0.023910,0.064474,0.395085,0.104568
3,3,2019-08-31,0.068414,0.017717,0.064474,0.395085,0.104568
4,4,2019-09-01,0.070283,0.017690,0.064474,0.395085,0.104568


In [51]:
Merged_Final.head()

,Date,LTC_Price,LTC_Volume,ETH_Price,ETH_Volume,BTC_Price,BTC_Volume,Gold_Price,Interest_Rate,VIX_Close
0,2019-08-28,0.102652,0.154545,0.013460,0.056006,0.070231,0.035507,0.079985,0.393195,0.109768
1,2019-08-29,0.093292,0.166255,0.012530,0.061260,0.066646,0.033893,0.071482,0.393195,0.089108
2,2019-08-30,0.093953,0.126259,0.012385,0.044840,0.067937,0.023910,0.064474,0.395085,0.104568
3,2019-08-31,0.094637,0.120992,0.013158,0.045492,0.068414,0.017717,0.064474,0.395085,0.104568
4,2019-09-01,0.099049,0.117904,0.012980,0.042150,0.070283,0.017690,0.064474,0.395085,0.104568


In [50]:
Merged_Final = Merged_Final.drop(columns=['Unnamed: 0'])

In [52]:
Merged_Final.to_csv('Merged_Final.csv', index=False)